In [1]:
import pandas as pd
import numpy as np

In [4]:
workers_answers = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_answers.csv")
workers_questionnaire = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_questionnaire.csv")

In [5]:
absolute_frequencies = workers_questionnaire['question_answer_text'].value_counts()
relative_frequencies = absolute_frequencies / len(workers_questionnaire)

average_confidence = workers_answers.groupby('document_index')['Confidence_value'].mean()
average_time_elapsed = workers_answers.groupby('document_index')['time_elapsed'].mean()

In [ ]:
absolute_frequencies

question_answer_text
Super Mario                                                  12
18-24                                                        11
Male                                                         10
Quite inclined (yes, they influence my purchase decision)     7
A few times a week                                            6
Rarely (for 20% of products)                                  6
Never                                                         2
Often (for 80% of products)                                   2
Female                                                        2
Occasionally (for 50% of products)                            2
Very inclined (yes, I decisively base my choices)             2
Not very inclined (no, I consider other factors)              2
Rarely                                                        2
Never (I'm not interested in writing reviews)                 2
Every day                                                     1
25-34              